<a href="https://colab.research.google.com/github/mohamedamrfarouk/arabic_dialect_classifier/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import tha libraries

In [1]:
import numpy as np
import pandas as pd
import json
import re
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import string
import pickle

In [2]:
vocab_size = 100000
embedding_dim = 64
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

[dataset with the text](https://drive.google.com/file/d/12Lofn3NLtNCe6SeAfUer1YYBPs6TlZMl/view?usp=sharing)

# load the CSV file that has the text

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/dialect_dataset_with_text.csv",lineterminator='\n')
df.head(5)

,id,dialect,text
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


# clean the text

## normalize the text

In [5]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

## remove any punctuation

In [6]:
def remove_any_punctuation(text):
    punctuations = string.punctuation
    for ele in text:
      if ele in punctuations:
        text = text.replace(ele, "")
    return text

## remove the hashtags

In [7]:
def remove_hashtags_mentions(text):
    text = re.sub("@([a-zA-Z0-9_]{1,50})","",text)
    text = re.sub("#([a-zA-Z0-9_]{1,50})","",text)
    return text

## remove the emojies

In [8]:
def remove_emojies(text):
  RE_EMOJI = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
  text = re.sub(RE_EMOJI,"",text)
  return text

## clean the text

In [9]:
def clean_the_text(text):
  text = str(text)
  text = normalize_arabic(text)
  text = remove_hashtags_mentions(text)
  text = remove_any_punctuation(text)
  text = remove_emojies(text)
  text = text.strip()
  return  text

# apply the cleaning function text on every record

In [10]:
df["text"] = df.text.apply(clean_the_text)
df = df.drop(['id'], axis=1)

# save the cleaned data frame to be used at any time

In [11]:
df.to_csv('/content/clean_data.csv')

In [12]:
def tokenize_data(X):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(X)
    return tokenizer


In [13]:
def save_tokenizer_file(tokenizer):
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
tokenizer = tokenize_data(df.text)
save_tokenizer_file(tokenizer)